# Import and loading

In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split

In [44]:
df = pd.read_csv('coords.csv')

# 전처리

## Visibility와 상관계수 확인 후 전처리

In [45]:
#상관계수 확인 후, visibility가 소용없는 정보라는 것을 알게 되었음. 제거 ㄱㄱ
df = df.filter(regex='^x[0-9]$|^y[0-9]$|^z[0-9]$|^x1[0-3]|^y1[0-3]|^z1[0-3]|^c')
df

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z10,x11,y11,z11,x12,y12,z12,x13,y13,z13
0,1,0.574625,0.645155,-2.716616,0.613432,0.541128,-2.666490,0.639249,0.540640,-2.666276,...,-2.451602,0.514353,0.742452,-2.478498,0.843735,0.930335,-1.423426,0.215217,0.909538,-1.495106
1,0,0.553470,0.509528,-0.966766,0.578971,0.449093,-0.893713,0.593508,0.450831,-0.893681,...,-0.799250,0.514640,0.574111,-0.822672,0.745233,0.819062,-0.158500,0.303293,0.803124,-0.380174
2,0,0.558947,0.515051,-0.976771,0.586876,0.458080,-0.891556,0.602352,0.459791,-0.891732,...,-0.807065,0.521150,0.585842,-0.821311,0.744071,0.845813,-0.142901,0.326350,0.820609,-0.297953
3,0,0.514702,0.311331,-1.469513,0.542640,0.250292,-1.378310,0.562116,0.251408,-1.378602,...,-1.269864,0.467532,0.402337,-1.283544,0.762135,0.741428,-0.504047,0.223045,0.726528,-0.643348
4,0,0.549252,0.420117,-1.490885,0.575709,0.350068,-1.412757,0.590431,0.351519,-1.413012,...,-1.292186,0.507645,0.498190,-1.302849,0.799121,0.781267,-0.517553,0.288866,0.790666,-0.664796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0,0.558450,0.544843,-1.286351,0.583285,0.483451,-1.209068,0.600140,0.485270,-1.209583,...,-1.089105,0.518663,0.630980,-1.105678,0.758486,0.893598,-0.347194,0.312175,0.883522,-0.417397
162,1,0.564978,0.556759,-2.624220,0.615437,0.454106,-2.599663,0.645236,0.454949,-2.599093,...,-2.399876,0.500437,0.651872,-2.425028,0.821074,0.784820,-1.420461,0.219333,0.817500,-1.529092
163,0,0.587629,0.544794,-1.301673,0.613657,0.485449,-1.213384,0.631125,0.487480,-1.213808,...,-1.112926,0.547139,0.623377,-1.133447,0.778429,0.881764,-0.362578,0.346102,0.888308,-0.419500
164,1,0.577460,0.603736,-2.596430,0.639357,0.490256,-2.607297,0.668227,0.491690,-2.606560,...,-2.398801,0.514967,0.680564,-2.405980,0.838056,0.841274,-1.644524,0.267406,0.766791,-1.468095


## 각도 정보 추가

In [78]:
def angle3(a,b,c): #세 점 사이의 각도를 구한다.
    a_x= np.array(df['x{}'.format(a)]) #First
    a_y = np.array(df['y{}'.format(a)])

    b_x = np.array(df['x{}'.format(b)]) #Mid
    b_y = np.array(df['y{}'.format(b)])

    c_x = np.array(df['x{}'.format(c)])
    c_y = np.array(df['y{}'.format(c)]) #End

    #y from endpoint - y form midpoint, x form end - x from mind
    radians=np.arctan2(c_y-b_y,c_x-b_x) - np.arctan2(a_y-b_y,a_x-b_x)
    angle=np.abs(radians*180.0/np.pi)

    for i in range(0,angle.shape[0]):
        if angle[i]>180.0:
            angle[i] = 360-angle[i]

    return pd.Series(angle)

In [47]:
#left angle 추가 -> csv에서는 1부터 시작. landmarks에서는 0부터 시작.
left_angle = angle3(2,4,8,df)

#right angle 추가
right_angle = angle3(5,7,9,df)

df['left_angle'] = left_angle
df['right_angle'] = right_angle

df

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,y11,z11,x12,y12,z12,x13,y13,z13,left_angle,right_angle
0,1,0.574625,0.645155,-2.716616,0.613432,0.541128,-2.666490,0.639249,0.540640,-2.666276,...,0.742452,-2.478498,0.843735,0.930335,-1.423426,0.215217,0.909538,-1.495106,150.614971,170.028614
1,0,0.553470,0.509528,-0.966766,0.578971,0.449093,-0.893713,0.593508,0.450831,-0.893681,...,0.574111,-0.822672,0.745233,0.819062,-0.158500,0.303293,0.803124,-0.380174,135.733230,145.717484
2,0,0.558947,0.515051,-0.976771,0.586876,0.458080,-0.891556,0.602352,0.459791,-0.891732,...,0.585842,-0.821311,0.744071,0.845813,-0.142901,0.326350,0.820609,-0.297953,127.860083,133.029917
3,0,0.514702,0.311331,-1.469513,0.542640,0.250292,-1.378310,0.562116,0.251408,-1.378602,...,0.402337,-1.283544,0.762135,0.741428,-0.504047,0.223045,0.726528,-0.643348,124.573389,135.326580
4,0,0.549252,0.420117,-1.490885,0.575709,0.350068,-1.412757,0.590431,0.351519,-1.413012,...,0.498190,-1.302849,0.799121,0.781267,-0.517553,0.288866,0.790666,-0.664796,130.939040,137.334540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0,0.558450,0.544843,-1.286351,0.583285,0.483451,-1.209068,0.600140,0.485270,-1.209583,...,0.630980,-1.105678,0.758486,0.893598,-0.347194,0.312175,0.883522,-0.417397,129.958664,127.620171
162,1,0.564978,0.556759,-2.624220,0.615437,0.454106,-2.599663,0.645236,0.454949,-2.599093,...,0.651872,-2.425028,0.821074,0.784820,-1.420461,0.219333,0.817500,-1.529092,156.379412,160.745467
163,0,0.587629,0.544794,-1.301673,0.613657,0.485449,-1.213384,0.631125,0.487480,-1.213808,...,0.623377,-1.133447,0.778429,0.881764,-0.362578,0.346102,0.888308,-0.419500,132.106487,126.591478
164,1,0.577460,0.603736,-2.596430,0.639357,0.490256,-2.607297,0.668227,0.491690,-2.606560,...,0.680564,-2.405980,0.838056,0.841274,-1.644524,0.267406,0.766791,-1.468095,154.659531,154.707187


# Train

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [50]:
X = df.drop('class',axis=1) #class를 제외한 입력 feature만 남김.
y = df['class']

In [51]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123)

In [59]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [60]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(),LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [61]:
fit_models={}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [63]:
yhat = fit_models['rf'].predict(X_test)

In [64]:
with open('Textneck.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 실시간 판단 Model

In [95]:
def angle3(a,b,c,df): #세 점 사이의 각도를 구한다.
    a_x= np.array(df['x{}'.format(a)]) #First
    a_y = np.array(df['y{}'.format(a)])

    b_x = np.array(df['x{}'.format(b)]) #Mid
    b_y = np.array(df['y{}'.format(b)])

    c_x = np.array(df['x{}'.format(c)])
    c_y = np.array(df['y{}'.format(c)]) #End

    #y from endpoint - y form midpoint, x form end - x from mind
    radians=np.arctan2(c_y-b_y,c_x-b_x) - np.arctan2(a_y-b_y,a_x-b_x)
    angle=np.abs(radians*180.0/np.pi)

    for i in range(0,angle.shape[0]):
        if angle[i]>180.0:
            angle[i] = 360-angle[i]

    return pd.Series(angle)

In [67]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

In [66]:
with open('Textneck.pkl','rb') as f:
    model = pickle.load(f)

In [71]:
landmarks=['class']
for val in range(1,13+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val)]

In [98]:
#VISUALIZE DEGREE
cap =cv2.VideoCapture(0) #setting video capture device(number은 웹캠을 대표하는 숫자)

# Curl counter variables
counter = 0

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret, frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환
        
         #Extract landmarks
        try:  
            row = []
            for k in range(12+1):
                row.append(results.pose_landmarks.landmark[k].x)
                row.append(results.pose_landmarks.landmark[k].y)
                row.append(results.pose_landmarks.landmark[k].z)
            
            row = np.array(row).flatten()
            
            X = pd.DataFrame([row],columns=landmarks[1:])
            
            #각도들도 넣어주기
            #left angle 추가 -> csv에서는 1부터 시작. landmarks에서는 0부터 시작.
            left_angle = angle3(2,4,8,X)

            #right angle 추가
            right_angle = angle3(5,7,9,X)

            X['left_angle'] = left_angle
            X['right_angle'] = right_angle

            #predict
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 0 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Good'
            elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmax()] >= .7 :
                current_stage = 'Textneck'
                counter +=1
        
            #Setup status bow
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1) 


            #지금 상태
            cv2.putText(image,'CLASS',(15,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image, 'Textneck' if body_language_class == 1 else 'Good',(15,40)
                        ,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Stage data 
            cv2.putText(image,'Count',(180,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    
            cv2.putText(image,str(counter),(175,40),
                       cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Render detections
            #drawing 툴을 사용해서 감지된 자세포인트와 이들 간의 포인트를 연결해 보여준다.
            #mp_drawing.DrawingSpec은 관절부위와 bone 부분의 색깔, 굵기 등을 지정해준다.
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                     mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        except: #error가 있으면 실행x
            pass
        
        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.